<a href="https://colab.research.google.com/github/SaqlainHussainShah/Angular/blob/master/K_fold_Separate_Notebook_B_6k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps for implementation
#### Reading CSV
#### Data Description
#### Data Cleaning
#### Data Normalization
#### Features
#### Applying KNN and NB
#### Confusion Matrix

In [0]:
#Importing Necessary Modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score

### Reading CSV

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
df1=pd.read_csv("/content/drive/My Drive/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("/content/drive/My Drive/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("/content/drive/My Drive/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("/content/drive/My Drive/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("/content/drive/My Drive/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("/content/drive/My Drive/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("/content/drive/My Drive/Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("/content/drive/My Drive/Wednesday-workingHours.pcap_ISCX.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

In [0]:
del df1,df2,df3,df4,df5,df6,df7,df8


In [0]:
# category B attacks
df_brute_force=df[df[' Label']=='Web Attack-Brute Force']
df_xss=df[df[' Label']=='Web Attack-XSS']
df_sql=df[df[' Label']=='Web Attack-Sql Injection']
#df_heartbleed=df[df[' Label']=='Heartbleed']
df_slowloris=df[df[' Label']=='DoS slowloris']
df_portscan=df[df[' Label']=='PortScan']
df_slow_http_test=df[df[' Label']=='DoS Slowhttptest']

In [0]:
# benign data
df_benign=df[df[' Label']=='BENIGN']

In [10]:
df_cat_B=pd.concat([df_brute_force,df_xss,df_sql,df_slowloris,df_portscan,df_slow_http_test, df_benign])
df_cat_B.groupby(' Label').first()


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BENIGN,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.000000,0.000000,4000000,666666.6667,3.000000e+00,0.000000e+00,3.0,3.0,3.0,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,...,6.0,6.0,6.000000,0.000000,0.00000,0,0,0,0,1,0,0,0,0.0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS Slowhttptest,172.16.0.1-192.168.10.50-33372-80-6,172.16.0.1,33372,192.168.10.50,80,6,5/7/2017 10:15,16045,5,3,356.0,1964.0,356.0,0.0,71.200000,159.208040,1964.0,0.0,654.666667,1133.915929,144593.3313,498.597694,2.292143e+03,2.236413e+03,5415.0,73.0,16045.0,4.011250e+03,1.779768e+03,5415.0,1459.0,9788.0,4.894000e+03,9.461089e+02,5563.0,4225.0,0,0,0,0,...,0.0,1964.0,257.777778,650.575472,423248.44440,0,0,0,1,0,0,0,0,0.0,290.000000,71.200000,654.666667,168,0,0,0,0,0,0,5,356,3,1964,29200,235,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS slowloris,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,5169956,8,6,1101.0,4222.0,410.0,0.0,137.625000,185.758628,3525.0,0.0,703.666667,1395.868284,1029.602573,2.707953414,3.976889e+05,1.368409e+06,4951173.0,112.0,218783.0,3.125471e+04,3.068629e+04,78311.0,219.0,5169833.0,1.033967e+06,2.218975e+06,5002421.0,840.0,0,0,0,0,...,0.0,3525.0,354.866667,895.115146,801231.12380,0,0,0,1,0,0,0,0,0.0,380.214286,137.625000,703.666667,172,0,0,0,0,0,0,8,1101,6,4222,8192,254,7,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PortScan,172.16.0.1-192.168.10.50-40620-80-6,172.16.0.1,40620,192.168.10.50,80,6,7/7/2017 1:05,5021059,6,5,703.0,1414.0,356.0,0.0,117.166667,181.536130,1050.0,0.0,282.800000,456.923626,421.6242032,2.190772903,5.021059e+05,1.568379e+06,4965658.0,19.0,55401.0,1.108020e+04,1.761267e+04,41863.0,19.0,5020928.0,1.255232e+06,2.499940e+06,5005133.0,1053.0,0,0,0,0,...,0.0,1050.0,176.416667,317.471103,100787.90150,0,0,0,1,0,0,0,0,0.0,192.454545,117.166667,282.800000,200,0,0,0,0,0,0,6,703,5,1414,29200,243,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Web Attack-Brute Force,172.16.0.1-192.168.10.50-44380-80-6,172.16.0.1,44380,192.168.10.50,80,6,6/7/2017 9:15,5185118,7,7,1022.0,2321.0,372.0,0.0,146.000000,184.078788,1047.0,0.0,331.571429,439.659284,644.7297824,2.700034985,3.988552e+05,1.372181e+06,4963956.0,4.0,221162.0,3.686033e+04,5.614102e+04,141434.0,4.0,5185004.0,8.641673e+05,2.027593e+06,5001548.0,879.0,0,0,0,0,...,0.0,1047.0,222.866667,331.323939,109775.55240,0,0,0,1,0,0,0,0,1.0,238.785714,146.000000,331.571429,232,0,0,0,0,0,0,7,1022,7,2321,29200,252,3,32,0.0,0.0,0

In [0]:
del df_brute_force,df_xss,df_sql,df_slowloris,df_portscan,df_slow_http_test, df_benign

In [0]:
df=df_cat_B.copy()

In [0]:
del df_cat_B

In [14]:
df.groupby(' Label').first()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BENIGN,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.000000,0.000000,4000000,666666.6667,3.000000e+00,0.000000e+00,3.0,3.0,3.0,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,...,6.0,6.0,6.000000,0.000000,0.00000,0,0,0,0,1,0,0,0,0.0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS Slowhttptest,172.16.0.1-192.168.10.50-33372-80-6,172.16.0.1,33372,192.168.10.50,80,6,5/7/2017 10:15,16045,5,3,356.0,1964.0,356.0,0.0,71.200000,159.208040,1964.0,0.0,654.666667,1133.915929,144593.3313,498.597694,2.292143e+03,2.236413e+03,5415.0,73.0,16045.0,4.011250e+03,1.779768e+03,5415.0,1459.0,9788.0,4.894000e+03,9.461089e+02,5563.0,4225.0,0,0,0,0,...,0.0,1964.0,257.777778,650.575472,423248.44440,0,0,0,1,0,0,0,0,0.0,290.000000,71.200000,654.666667,168,0,0,0,0,0,0,5,356,3,1964,29200,235,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS slowloris,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,5169956,8,6,1101.0,4222.0,410.0,0.0,137.625000,185.758628,3525.0,0.0,703.666667,1395.868284,1029.602573,2.707953414,3.976889e+05,1.368409e+06,4951173.0,112.0,218783.0,3.125471e+04,3.068629e+04,78311.0,219.0,5169833.0,1.033967e+06,2.218975e+06,5002421.0,840.0,0,0,0,0,...,0.0,3525.0,354.866667,895.115146,801231.12380,0,0,0,1,0,0,0,0,0.0,380.214286,137.625000,703.666667,172,0,0,0,0,0,0,8,1101,6,4222,8192,254,7,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PortScan,172.16.0.1-192.168.10.50-40620-80-6,172.16.0.1,40620,192.168.10.50,80,6,7/7/2017 1:05,5021059,6,5,703.0,1414.0,356.0,0.0,117.166667,181.536130,1050.0,0.0,282.800000,456.923626,421.6242032,2.190772903,5.021059e+05,1.568379e+06,4965658.0,19.0,55401.0,1.108020e+04,1.761267e+04,41863.0,19.0,5020928.0,1.255232e+06,2.499940e+06,5005133.0,1053.0,0,0,0,0,...,0.0,1050.0,176.416667,317.471103,100787.90150,0,0,0,1,0,0,0,0,0.0,192.454545,117.166667,282.800000,200,0,0,0,0,0,0,6,703,5,1414,29200,243,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Web Attack-Brute Force,172.16.0.1-192.168.10.50-44380-80-6,172.16.0.1,44380,192.168.10.50,80,6,6/7/2017 9:15,5185118,7,7,1022.0,2321.0,372.0,0.0,146.000000,184.078788,1047.0,0.0,331.571429,439.659284,644.7297824,2.700034985,3.988552e+05,1.372181e+06,4963956.0,4.0,221162.0,3.686033e+04,5.614102e+04,141434.0,4.0,5185004.0,8.641673e+05,2.027593e+06,5001548.0,879.0,0,0,0,0,...,0.0,1047.0,222.866667,331.323939,109775.55240,0,0,0,1,0,0,0,0,1.0,238.785714,146.000000,331.571429,232,0,0,0,0,0,0,7,1022,7,2321,29200,252,3,32,0.0,0.0,0

### Data Description

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2445502 entries, 12637 to 692702
Data columns (total 85 columns):
Flow ID                         object
 Source IP                      object
 Source Port                    int64
 Destination IP                 object
 Destination Port               int64
 Protocol                       int64
 Timestamp                      object
 Flow Duration                  int64
 Total Fwd Packets              int64
 Total Backward Packets         int64
Total Length of Fwd Packets     float64
 Total Length of Bwd Packets    float64
 Fwd Packet Length Max          float64
 Fwd Packet Length Min          float64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max           float64
 Bwd Packet Length Min          float64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
Flow Bytes/s                    object
 Flow Packets/s                 object
 Flow IAT Mean        

In [16]:
df.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2445502.0,2.445502e+06,2445502.0,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2445502.0,2445502.0,2445502.0,2445502.0,2445502.0,2445502.0,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06,2.445502e+06
mean,4.025521e+04,9.316412e+03,1.049161e+01,1.070480e+07,1.000689e+01,1.135899e+01,5.954642e+02,1.752437e+04,2.150081e+02,2.152577e+01,6.229543e+01,6.954472e+01,3.695331e+02,4.747930e+01,1.500813e+02,1.147316e+02,8.936951e+05,1.547043e+06,4.351347e+06,1.401210e+05,1.040628e+07,1.860810e+06,1.215960e+06,4.209320e+06,1.119952e+06,9.628262e+06,1.771238e+06,9.793314e+05,3.638800e+06,1.115493e+06,5.213531e-02,0.0,1.288079e-04,0.0,-3.011647e+04,-2.651612e+03,5.637947e+04,8.009377e+03,1.890680e+01,4.610986e+02,1.022395e+02,1.381271e+02,1.080328e+05,1.684235e-02,5.213531e-02,2.805150e-04,3.089075e-01,2.681208e-01,1.081909e-01,1.288079e-04,2.817417e-04,7.174646e-01,1.162059e+02,6.229543e+01,1.500813e+02,-3.011647e+04,0.0,0.0,0.0,0.0,0.0,0.0,1.000689e+01,5.954521e+02,1.135899e+01,1.752398e+04,7.519901e+03,2.273980e+03,5.928044e+00,-3.177567e+03,8.267207e+04,4.747196e+04,1.654826e+05,5.585384e+04,3.618246e+06,2.716610e+05,3.837364e+06,3.380233e+06
std,2.344245e+04,1.936528e+04,5.413308e+00,2.949875e+07,8.065138e+02,1.073044e+03,1.025727e+04,2.434769e+06,7.653926e+02,6.446373e+01,1.986013e+02,3.007253e+02,7.906874e+02,7.169981e+01,2.766348e+02,2.678298e+02,4.087446e+06,5.911421e+06,1.446538e+07,2.595454e+06,2.937080e+07,9.286063e+06,4.692548e+06,1.454386e+07,8.971274e+06,2.850742e+07,9.320263e+06,4.386670e+06,1.380373e+07,8.918392e+06,2.222999e-01,0.0,1.134863e-02,0.0,2.265047e+07,1.562411e+06,2.243329e+05,4.067556e+04,2.630122e+01,1.043601e+03,1.827199e+02,2.982540e+02,5.598068e+05,1.286806e-01,2.222999e-01,1.674624e-02,4.620430e-01,4.429810e-01,3.106214e-01,1.134863e-02,1.678280e-02,6.806215e-01,1.931817e+02,1.986013e+02,2.766348e+02,2.265047e+07,0.0,0.0,0.0,0.0,0.0,0.0,8.065138e+02,1.02

In [17]:
df.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

### Data Cleaning

In [18]:
df2=df[df.columns[7:-1]]
df2[' Label']=df[' Label']
df=df2
df2=[]
del df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
len(df.columns)

78

In [0]:
df=df.dropna( axis=0, how='any')
df=df.replace(',,', np.nan, inplace=False)
df=df.drop(columns=[' Fwd Header Length.1'], axis=1, inplace=False)

In [0]:
df.replace("Infinity", 0, inplace=True)
df['Flow Bytes/s'].replace("Infinity", 0,inplace=True)
df[" Flow Packets/s"].replace("Infinity", 0, inplace=True)
df[" Flow Packets/s"].replace(np.nan, 0, inplace=True)
df['Flow Bytes/s'].replace(np.nan, 0,inplace=True)


df["Bwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)

df[" Bwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df[" Bwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df["Fwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" CWE Flag Count"].replace("Infinity", 0, inplace=True)
df[" CWE Flag Count"].replace(",,", 0, inplace=True)
df[" CWE Flag Count"].replace(np.nan, 0, inplace=True)

df[" Bwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd URG Flags"].replace(",,", 0, inplace=True)
df[" Bwd URG Flags"].replace(np.nan, 0, inplace=True)

df[" Bwd PSH Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd PSH Flags"].replace(",,", 0, inplace=True)
df[" Bwd PSH Flags"].replace(np.nan, 0, inplace=True)

df[" Fwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Fwd URG Flags"].replace(",,", 0, inplace=True)
df[" Fwd URG Flags"].replace(np.nan, 0, inplace=True)

In [0]:
df["Flow Bytes/s"]=df["Flow Bytes/s"].astype("float64")
df[' Flow Packets/s']=df[" Flow Packets/s"].astype("float64")

In [0]:
df['Bwd Avg Bulk Rate']=df["Bwd Avg Bulk Rate"].astype("float64")
df[' Bwd Avg Packets/Bulk']=df[" Bwd Avg Packets/Bulk"].astype("float64")
df[' Bwd Avg Bytes/Bulk']=df[" Bwd Avg Bytes/Bulk"].astype("float64")
df[' Fwd Avg Bulk Rate']=df[" Fwd Avg Bulk Rate"].astype("float64")
df[' Fwd Avg Packets/Bulk']=df[" Fwd Avg Packets/Bulk"].astype("float64")
df['Fwd Avg Bytes/Bulk']=df["Fwd Avg Bytes/Bulk"].astype("float64")
df[' CWE Flag Count']=df[" CWE Flag Count"].astype("float64")
df[' Bwd URG Flags']=df[" Bwd URG Flags"].astype("float64")
df[' Bwd PSH Flags']=df[" Bwd PSH Flags"].astype("float64")
df[' Fwd URG Flags']=df[" Fwd URG Flags"].astype("float64")

In [0]:
pd.set_option('display.max_rows', df.shape[0])

In [0]:
df.replace('Infinity',0.0, inplace=True)

In [0]:
df.replace('NaN',0.0, inplace=True)

In [27]:
df.columns

Index([' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
       'FIN Flag Count', ' SY

In [0]:



X=df[df.columns[0:-1]]
y=df[df.columns[-1]]

del df

### Data Normalization

In [0]:
from scipy import stats

In [30]:


cols = list(X.columns)
for col in cols:
    X[col] = stats.zscore(X[col])



/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2315: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [31]:
X.head()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
12637,-0.187163,-0.003730,-0.004064,0.041571,-0.006245,0.205050,-0.333952,0.421386,0.380791,0.856674,-0.662275,0.655935,1.213033,-0.061525,-0.276008,-0.121090,-0.029622,0.042297,-0.053990,-0.346811,-0.196436,-0.247186,-0.279725,-0.124848,-0.155908,-0.097346,0.238909,0.098671,-0.124990,-0.234548,NaN,-0.011351,NaN,0.001340,0.001846,-0.251337,-0.196893,-0.718948,0.561311,0.660044,0.647640,0.003081,-0.130818,-0.234548,-0.016752,1.495551,-0.605105,-0.348278,-0.011351,-0.016789,0.415044,0.634397,0.421386,0.655935,NaN,NaN,NaN,NaN,NaN,NaN,-0.003730,0.041634,-0.004064,-0.006245,1.441549,-0.223150,-0.004278,0.00275,-0.123421,-0.112549,-0.152547,-0.094681,-0.269685,-0.091926,-0.272536,-0.257564
12643,-0.191493,-0.000010,-0.004064,0.013690,-0.003141,0.235098,-0.333952,0.056864,0.286376,5.026192,-0.662275,4.559253,5.694141,-0.061473,-0.276005,-0.141338,-0.052106,0.042239,-0.053978,-0.182163,-0.139896,0.092558,0.051777,-0.124827,-0.335807,-0.189054,-0.220863,-0.262067,-0.125084,-0.234548,NaN,-0.011351,NaN,0.001344,0.001846,-0.251334,-0.196892,-0.718948,3.720354,2.667981,3.527983,2.338351,-0.130818,-0.234548,-0.016752,1.495551,-0.605105,-0.348278,-0.011351,-0.016789,-1.054160,2.630779,0.056864,4.559253,NaN,NaN,NaN,NaN,NaN,NaN,-0.000010,0.013712,-0.004064,-0.003140,1.441549,-0.224144,-0.005738,0.00275,-0.123421,-0.112549,-0.152547,-0.094681,-0.269685,-0.091926,-0.272536,-0.257564
12700,-0.362922,-0.011169,-0.009655,-0.058058,-0.007198,-0.280938,-0.333952,-0.313700,-0.231277,-0.467405,-0.662275,-0.542584,-0.428417,-0.061548,-0.170042,-0.218643,-0.261728,-0.300833,-0.053960,-0.354341,-0.200405,-0.259149,-0.289449,-0.124848,-0.337777,-0.190058,-0.223271,-0.263633,-0.125088,-0.234548,NaN,-0.011351,NaN,0.001331,0.001718,-0.196314,0.106566,-0.718948,-0.441879,-0.559604,-0.463166,-0.192999,-0.130818,-0.234548,-0.016752,-0.668650,1.652605,2.871267,-0.011351,-0.016789,0.415044,-0.601605,-0.313700,-0.542584,NaN,NaN,NaN,NaN,NaN,NaN,-0.011169,-0.058143,-0.009655,-0.007198,-0.482015,-0.223150,-0.008659,0.00275,-0.123421,-0.112549,-0.152547,-0.094681,-0.269685,-0.091926,-0.272536,-0.257564
12712,-0.184248,-0.003730,-0.005927,0.036404,-0.006611,0.451963,-0.333952,0.383265,0.568400,0.880702,-0.662275,0.491209,1.302588,-0.061532,-0.276010,-0.101437,-0.009889,0.042155,-0.053974,-0.174886,-0.105806,0.167687,0.051694,-0.124812,-0.328300,-0.182811,-0.193273,-0.244440,-0.124973,-0.234548,NaN,-0.011351,NaN,0.001340,0.001805,-0.251337,-0.196903,-0.718948,0.579516,0.450291,0.643425,0.001596,-0.130818,-0.234548,-0.016752,1.495551,-0.605105,-0.348278,-0.011351,-0.016789,-1.054160,0.433203,0.383265,0.491209,NaN,NaN,NaN,NaN,NaN,NaN,-0.003730,0.036459,-0.005927,-0.006611,1.441549,-0.224033,-0.005738,0.00275,-0.123421,-0.112549,-

### Feature Selection

In [0]:
features=[" Fwd Packet Length Max"," Flow IAT Std"," Fwd Packet Length Std" ,"Fwd IAT Total",' Flow Packets/s',' Flow Duration', " Fwd Packet Length Mean", " Total Length of Bwd Packets", "Flow Bytes/s", "Total Length of Fwd Packets", " Flow IAT Mean", " Bwd Packet Length Mean",  " Flow IAT Max", " Bwd Packet Length Std", ' Total Fwd Packets', ' Total Backward Packets']

In [33]:
X=X[features].copy()
X.head()

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Packets/s,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Flow Bytes/s,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets
12637,0.205050,-0.029622,0.380791,-0.346811,-0.276008,-0.187163,0.421386,-0.006245,-0.061525,0.041571,-0.121090,0.655935,0.042297,1.213033,-0.003730,-0.004064
12643,0.235098,-0.052106,0.286376,-0.182163,-0.276005,-0.191493,0.056864,-0.003141,-0.061473,0.013690,-0.141338,4.559253,0.042239,5.694141,-0.000010,-0.004064
12700,-0.280938,-0.261728,-0.231277,-0.354341,-0.170042,-0.362922,-0.313700,-0.007198,-0.061548,-0.058058,-0.218643,-0.542584,-0.300833,-0.428417,-0.011169,-0.009655
12712,0.451963,-0.009889,0.568400,-0.174886,-0.276010,-0.184248,0.383265,-0.006611,-0.061532,0.036404,-0.101437,0.491209,0.042155,1.302588,-0.003730,-0.005927
12791,0.457189,0.003389,0.572301,-0.183413,-0.276010,-0.192738,0.386142,-0.006611,-0.061531,0.036794,-0.095842,0.749658,0.042207,1.447780,-0.003730,-0.006859


In [34]:
len(X.columns)

16

### Applying Algorithms

In [0]:
#Run if not using K-Fold cross validation
from sklearn.model_selection import train_test_split     # import module for train test split

In [0]:
#Run if not using K-Fold cross validation
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=10)

#### KNN

In [0]:
#Run if not using K-Fold cross validation
from sklearn.neighbors import KNeighborsClassifier

In [0]:
#Run if not using K-Fold cross validation
t1=time.time()

for i in range(1,len(X_train.columns)+1):
    knn=KNeighborsClassifier(n_neighbors=i)
    model_knn=knn.fit(X_train,y_train)
    yhat=model_knn.predict(X_test)
    print("for " , i,  " as K, accuracy is : ", accuracy_score(y_test, yhat))
t2=time.time()
print(" time for ", i ," k's :: ", (t2-t1)/60 , " minutes")

for  1  as K, accuracy is :  0.998781233449007
for  2  as K, accuracy is :  0.9987035268568297
for  3  as K, accuracy is :  0.9988364460276594
for  4  as K, accuracy is :  0.998807817283173
for  5  as K, accuracy is :  0.9988323562070185
for  6  as K, accuracy is :  0.9988200867450958
for  7  as K, accuracy is :  0.9988405358483003
for  8  as K, accuracy is :  0.9988303112966981
for  9  as K, accuracy is :  0.9988364460276594
for  10  as K, accuracy is :  0.9988180418347753
for  11  as K, accuracy is :  0.9987996376418912
for  12  as K, accuracy is :  0.998785323269648
for  13  as K, accuracy is :  0.9987607843458025
for  14  as K, accuracy is :  0.9987607843458025
for  15  as K, accuracy is :  0.9987260208703548
for  16  as K, accuracy is :  0.9987260208703548
 time for  16  k's ::  61.21009658972422  minutes


#### K Fold Cross Validation and KNN

In [0]:
df=X.copy()
df[' Label']=y.values

In [0]:
df.reset_index(inplace=True)

In [37]:
df2=df[df.columns[1:-1]]
df2[' Label']=df[df.columns[-1]]
df=df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
del df2

In [39]:
df.head()


,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Packets/s,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Flow Bytes/s,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets,Label
0,0.205050,-0.029622,0.380791,-0.346811,-0.276008,-0.187163,0.421386,-0.006245,-0.061525,0.041571,-0.121090,0.655935,0.042297,1.213033,-0.003730,-0.004064,Web Attack-Brute Force
1,0.235098,-0.052106,0.286376,-0.182163,-0.276005,-0.191493,0.056864,-0.003141,-0.061473,0.013690,-0.141338,4.559253,0.042239,5.694141,-0.000010,-0.004064,Web Attack-Brute Force
2,-0.280938,-0.261728,-0.231277,-0.354341,-0.170042,-0.362922,-0.313700,-0.007198,-0.061548,-0.058058,-0.218643,-0.542584,-0.300833,-0.428417,-0.011169,-0.009655,Web Attack-Brute Force
3,0.451963,-0.009889,0.568400,-0.174886,-0.276010,-0.184248,0.383265,-0.006611,-0.061532,0.036404,-0.101437,0.491209,0.042155,1.302588,-0.003730,-0.005927,Web Attack-Brute Force
4,0.457189,0.003389,0.572301,-0.183413,-0.276010,-0.192738,0.386142,-0.006611,-0.061531,0.036794,-0.095842,0.749658,0.042207,1.447780,-0.003730,-0.006859,Web Attack-Brute Force


In [0]:
X=df[df.columns[0:-1]]
y=df[df.columns[-1]]

del df


In [41]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445093 entries, 0 to 2445092
Data columns (total 16 columns):
 Fwd Packet Length Max          float64
 Flow IAT Std                   float64
 Fwd Packet Length Std          float64
Fwd IAT Total                   float64
 Flow Packets/s                 float64
 Flow Duration                  float64
 Fwd Packet Length Mean         float64
 Total Length of Bwd Packets    float64
Flow Bytes/s                    float64
Total Length of Fwd Packets     float64
 Flow IAT Mean                  float64
 Bwd Packet Length Mean         float64
 Flow IAT Max                   float64
 Bwd Packet Length Std          float64
 Total Fwd Packets              float64
 Total Backward Packets         float64
dtypes: float64(16)
memory usage: 298.5 MB


In [42]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
i=0
scores=np.zeros(10)
t1=time.time()
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  
  
   #  can use loop here
     
  knn=KNeighborsClassifier(n_neighbors=6)
  model_knn=knn.fit(X_train,y_train)
  yhat=model_knn.predict(X_test)
  
  scores[i]=accuracy_score(y_test, yhat)
  print("for " , i,  " as K, accuracy is : ", scores[i])
  i=i+1
  
  
#     X_train=train[train.columns[0:-1]]
#     y_train=train[train.columns[-1]]
    
#     X_test=test[test.columns[0:-1]]
#     y_test=test[test.columns[-1]]


t2=time.time()
print(" time for ", i ," cross validations :: ", (t2-t1)/60 , " minutes")
    
   
     
     
    
     
     
     
    

for  0  as K, accuracy is :  0.29489591427753464
for  1  as K, accuracy is :  0.9998159584475073
for  2  as K, accuracy is :  0.999541941024907
for  3  as K, accuracy is :  0.9998282271818215
for  4  as K, accuracy is :  0.995619793136449
for  5  as K, accuracy is :  0.8766957453508869
for  6  as K, accuracy is :  0.9997341611147238
for  7  as K, accuracy is :  0.9990920579610566
for  8  as K, accuracy is :  0.999607376415592
for  9  as K, accuracy is :  0.9994192442813966
 time for  10  cross validations ::  91.98744525512059  minutes


In [43]:
print(" time in hours :::: ", (t2-t1)/(60*60), " hours")

 time in hours ::::  1.5331240875853434  hours


In [0]:
average_accuracy=scores.mean()
print("average accuracy of K fold KNN :: ", average_accuracy)

average accuracy of K fold KNN ::  0.9164250419191875


#### Save Model

In [0]:
import pickle
filename = 'drive/My Drive/model_cross_validation_cat_B_6k.sav'

In [0]:
pickle.dump(model_knn, open(filename, 'wb'))

In [0]:
# to load model
model_knn = pickle.load(open(filename, 'rb'))
y_hat=model_knn.predict(X_test)

#### Naive Bayes

##### Gaussian Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB

In [47]:
t1=time.time()

gnb = GaussianNB()
model_gnb = gnb.fit(X_train, y_train)
yhat_gnb=model_gnb.predict(X_test)

print("Accuracy :: ", accuracy_score(y_test,yhat))

t2=time.time()
print(" time for Naive Baye's :: ", (t2-t1) , " seconds")

Accuracy ::  0.9994192442813966
 time for Naive Baye's ::  4.847107410430908  seconds


##### Bernoulli Naive Bayes

In [0]:
from sklearn.naive_bayes import BernoulliNB

In [49]:


clf = BernoulliNB()
clf.fit(X_train, y_train)
yhat_bnb=clf.predict(X_test)
print("Accuracy ::: ", accuracy_score(y_test,yhat_bnb))



Accuracy :::  0.33908772274231214


### Confusion Matrix

In [50]:
y_test_arr=y_test.as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
true={}
false={}
not_detected={}

for x in range(len(y_test_arr)):
    if y_test_arr[x]==yhat[x]:
      if y_test_arr[x] in true:
        true[y_test_arr[x]]=true[y_test_arr[x]]+1
      else:
        true[y_test_arr[x]]=1
    elif y_test_arr[x]!=yhat[x]:
      if yhat[x] in false:
        false[yhat[x]]=false[yhat[x]]+1

        if y_test_arr[x] in not_detected:
          not_detected[y_test_arr[x]]=not_detected[y_test_arr[x]]+1
        else:
          not_detected[y_test_arr[x]]=1

      else:
        false[yhat[x]]=1

        if y_test_arr[x] in not_detected:
          not_detected[y_test_arr[x]]=not_detected[y_test_arr[x]]+1
        else:
          not_detected[y_test_arr[x]]=1

In [52]:
true

{'BENIGN': 244367}

In [53]:
false

{'DoS Slowhttptest': 88, 'DoS slowloris': 6, 'PortScan': 48}

In [54]:
not_detected

{'BENIGN': 142}